In [10]:
import boto3
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from folium import GeoJson
from io import BytesIO
import geopandas as gpd

In [7]:
import sys
sys.path.append("/agrilearn_app/agrilearn/submodules/commons/")

In [8]:
from agrilearn.crop_classification import evalutate_utils
from agrilearn.crop_classification import yaml_utils
from agrilearn.commons.s3 import s3_utils

### Global Variables

In [9]:
GEOPACKAGE_PATH = f"/agrilearn_app/datasets/base/geopackage/processed/SOYBEAN_96505_CORN_67593_SUGAR_CANE_35276_COTTON_6344_RICE_1842.gpkg"

# 1. Read Datasets

In [11]:
df = gpd.read_file(GEOPACKAGE_PATH)

In [13]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 207560 entries, 0 to 207559
Data columns (total 36 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   monitoring_class  207560 non-null  object        
 1   period            207560 non-null  object        
 2   fonte             207560 non-null  object        
 3   state             207560 non-null  object        
 4   area              207559 non-null  object        
 5   micro             134093 non-null  float64       
 6   start_season      207560 non-null  datetime64[ms]
 7   end_season        207560 non-null  datetime64[ms]
 8   peak_start        207560 non-null  datetime64[ms]
 9   peak_end          207560 non-null  datetime64[ms]
 10  eopath_location   207560 non-null  object        
 11  start_of_season   172284 non-null  object        
 12  peak_of_season    134093 non-null  datetime64[ms]
 13  end_of_season     172284 non-null  object        
 

In [14]:
df.shape

(207560, 36)

# 2 Split trianing and test 

In [15]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [16]:
val_test.shape

NameError: name 'val_test' is not defined

In [17]:
# train_, val_test = eval_util.split_dataset_by_state_and_strafity(gdf, 
#                                                                  label_state='state',
#                                                                  stratify_labels=['monitoring_class', 'period'],
#                                                                  train_size=0.7)

# val_, test_ = eval_util.split_dataset_by_state_and_strafity(val_test, 
#                                                             label_state='state',
#                                                             stratify_labels=['monitoring_class'],
#                                                             train_size=0.5)

# gdf['dataset_part'] = None
# gdf.loc[train_.index, 'dataset_part'] = 'train'
# gdf.loc[val_.index, 'dataset_part'] = 'val'
# gdf.loc[test_.index, 'dataset_part'] = 'test'

In [30]:
gdf['dataset_part'].value_counts()

dataset_part
train    26726
test      5737
val       5729
Name: count, dtype: int64

In [31]:
gdf['monitoring_class'].value_counts()

monitoring_class
SOYBEAN    29670
CORN        5710
COTTON      1639
RICE        1173
Name: count, dtype: int64

In [32]:
gdf['state'].value_counts()

state
MT    5709
SP    5079
PR    4911
MG    4380
RS    3338
GO    3322
MS    2542
MA    1781
TO    1571
BA    1193
PI    1158
RO    1066
PA     935
SC     710
RR     275
DF     156
AP      39
AC      27
Name: count, dtype: int64

In [33]:
print(f"There are {gdf.shape[0]} rows and {gdf.shape[1]} columns")

There are 38192 rows and 40 columns


In [34]:
print(f"Dados de {gdf['start_season'].min()} a {gdf['end_season'].max()}")

Dados de 2017-09-01 a 2022-11-01


In [35]:
gdf['dataset_part'].value_counts(normalize=True)*100

dataset_part
train    69.978006
test     15.021470
val      15.000524
Name: proportion, dtype: float64

In [38]:
gdf.to_file(GEOPACKAGE_OUTPUT,
           driver='GPKG', 
           engine='fiona')

# 3. Data Analysis

### 3.1 Analysing Target y (monitoring_class)

In [41]:
pd.concat([gdf['monitoring_class'].value_counts(),
          gdf['monitoring_class'].value_counts(normalize=True)*100], axis=1)

,count,proportion
monitoring_class,,
SOYBEAN,29670,77.686426
CORN,5710,14.950775
COTTON,1639,4.291475
RICE,1173,3.071324


### 3.2 Analysing dados por estado

In [42]:
pd.concat([gdf['state'].value_counts(),
          gdf['state'].value_counts(normalize=True)*100], axis=1)

,count,proportion
state,,
MT,5709,14.948157
SP,5079,13.298597
PR,4911,12.858714
MG,4380,11.468370
RS,3338,8.740050
GO,3322,8.698157
MS,2542,6.655844
MA,1781,4.663280
TO,1571,4.113427


### 3.3 Analysing dados por estado e classe de cultura

In [43]:
gdf.groupby(['state', 'monitoring_class']).agg({'period':'count'})

period
state monitoring_class        
AC    CORN                  27
AP    SOYBEAN               39
BA    CORN                 249
      COTTON               383
      SOYBEAN              561
DF    CORN                  61
      SOYBEAN               95
GO    CORN                 484
      COTTON               108
      RICE                  86
      SOYBEAN             2644
MA    CORN                 305
      COTTON                 9
      SOYBEAN             1467
MG    CORN                 997
      COTTON               188
      SOYBEAN             3195
MS    CORN                 211
      COTTON                70
      RICE                  67
      SOYBEAN             2194
MT    CORN                 365
      COTTON               798
      SOYBEAN             4546
PA    CORN                 109
      SOYBEAN              826
PI    CORN                 273
      COTTON                21
      SOYBEAN              864
PR    CORN                 965
      RICE                 122
      SOYBEAN             3824
RO    CORN                  22
      SOYBEAN             1044
RR    SOYBEAN              275
RS    CORN                 504
      RICE                 595
      SOYBEAN             2239
SC    CORN                 140
      RICE                 219
      SOYBEAN              351
SP    CORN                 862
      COTTON                21
      SOYBEAN             4196
TO    CORN                 136
      COTTON                41
      RICE                  84
      SOYBEAN             1310

### 3.4 Analysis field_id (existem IDs duplicados para os talhões)

In [44]:
gdf['field_id'].nunique()

36234

In [45]:
gdf[gdf.duplicated(['field_id','period'])]

,monitoring_class,period,fonte,cultura_2,state,area,meso,obs_extra,eopath_location,start_season,...,dataset_part,local_eopatch_path,total_imagens_do_intervalo,total_de_imagem,status_missing_images,days_gap,dates_diff,mean_dates_diff,status_series_missing,geometry
27054,SOYBEAN,2021/2022,Agrosatélite - Grãos Brasil 2021/2022,None,RO,551422.0,3,,start_2021-10-01_end_2022-05-01_monitoring_cla...,2021-10-01,...,train,/agrilearn_app/datasets/eopatchs/processed/SOY...,49,31,OK,None,None,5.17,OK,"POLYGON ((-62.38324 -10.12717, -62.38324 -10.1..."
27060,SOYBEAN,2021/2022,Agrosatélite - Grãos Brasil 2021/2022,None,RO,3148591.0,4,,start_2021-10-01_end_2022-05-01_monitoring_cla...,2021-10-01,...,train,/agrilearn_app/datasets/eopatchs/processed/SOY...,49,23,OK,None,None,5.23,OK,"POLYGON ((-62.01629 -11.33566, -62.01628 -11.3..."
27071,SOYBEAN,2021/2022,Agrosatélite - Grãos Brasil 2021/2022,None,RO,2135605.0,7,,start_2021-10-01_end_2022-05-01_monitoring_cla...,2021-10-01,...,train,/agrilearn_app/datasets/eopatchs/processed/SOY...,49,25,OK,None,None,5.00,OK,"POLYGON ((-60.58 -13.51871, -60.58 -13.51845, ..."
27077,SOYBEAN,2021/2022,Agrosatélite - Grãos Brasil 2021/2022,None,PA,568770.0,31,,start_2021-10-01_end_2022-09-01_monitoring_cla...,2021-10-01,...,val,/agrilearn_app/datasets/eopatchs/processed/SOY...,49,48,OK,None,None,2.51,OK,"POLYGON ((-54.4106 -2.77062, -54.41037 -2.7716..."
27095,SOYBEAN,2020/2021,Agrosatélite - Grãos Brasil 2020/2021,None,PA,1404261.0,42,,start_2020-10-01_end_2021-09-01_monitoring_cla...,2020-10-01,...,train,/agrilearn_app/datasets/eopatchs/processed/SOY...,49,26,OK,None,None,5.20,OK,"MULTIPOLYGON (((-48.91709 -3.4195, -48.91695 -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38147,RICE,2019/2020,Conab - Arroz Irrigado 2019/2020,None,RS,108551.0,488,,start_2019-09-01_end_2020-06-01_monitoring_cla...,2019-09-01,...,train,/agrilearn_app/datasets/eopatchs/processed/SOY...,55,28,OK,None,None,5.19,OK,"POLYGON ((-54.15517 -29.92234, -54.15512 -29.9..."
38151,RICE,2019/2020,Conab - Arroz Irrigado 2019/2020,None,RS,384161.0,493,,start_2019-09-01_end_2020-06-01_monitoring_cla...,2019-09-01,...,train,/agrilearn_app/datasets/eopatchs/processed/SOY...,55,28,OK,None,None,5.19,OK,"POLYGON ((-51.41222 -29.70816, -51.41241 -29.7..."
38165,RICE,2019/2020,Conab - Arroz Irrigado 2019/2020,None,RS,714262.0,505,,start_2019-09-01_end_2020-06-01_monitoring_cla...,2019-09-01,...,train,/agrilearn_app/datasets/eopatchs/processed/SOY...,68,34,OK,None,None,5.15,OK,"POLYGON ((-52.55275 -32.29614, -52.55275 -32.2..."
38169,RICE,2021/2022,Conab - Arroz Irrigado 2021/2022,None,GO,125860.0,551,,start_2021-10-01_end_2022-06-01_monitoring_cla...,2021-10-01,...,train,/agrilearn_app/datasets/eopatchs/processed/SOY...,68,28,OK,None,None,5.00,OK,"POLYGON ((-47.03756 -14.50845, -47.03755 -14.5..."


In [46]:
gdf[gdf.duplicated(['field_id','period'])]['field_id'].unique()

array(['9_2', '14_3', '23_1', '35_1', '40_8', '40_10', '41_5', '41_6',
       '83_1', '126_1', '134_2', '140_2', '142_0', '146_0', '157_1',
       '161_5', '180_1', '187_1', '201_3', '202_1', '210_1', '214_0',
       '216_3', '229_6', '237_1', '266_1', '275_2', '280_2', '281_2',
       '282_1', '309_2', '315_4', '315_6', '356_1', '363_0', '371_3',
       '372_4', '373_4', '397_0', '399_0', '416_1', '445_10', '460_1',
       '460_3', '470_1', '476_1', '757_2', '785_2', '801_3', '801_6',
       '803_11', '803_12', '844_2', '847_3', '849_4', '850_3', '850_4',
       '850_5', '854_4', '854_5', '859_4', '859_10', '859_16', '882_2',
       '940_1', '43_1', '74_3', '74_6', '113_2', '140_1', '443_2',
       '745_2', '847_2', '859_3', '859_12', '1_0', '1_2', '6_2', '8_3',
       '21_0', '48_4', '61_3', '61_7', '62_18', '78_0', '78_3', '89_2',
       '89_4', '92_4', '132_3', '147_9', '147_12', '148_3', '148_4',
       '148_9', '154_2', '170_3', '172_2', '185_1', '209_3', '219_2',
       '226_0',

In [47]:
#df[df['field_id'].isin(['0_6'])].to_csv('/agrilearn_app/datasets/cana-v1/csvs/duplicated_lines_CANA-sample-02.csv')

In [48]:
gdf[['cultura_2', 'monitoring_class']]

,cultura_2,monitoring_class
0,None,SOYBEAN
1,None,SOYBEAN
2,None,SOYBEAN
3,None,SOYBEAN
4,None,SOYBEAN
...,...,...
38187,None,RICE
38188,None,RICE
38189,None,RICE
38190,None,RICE


### 4.5 Analysis of period

In [50]:
gdf.groupby(['dataset_part', 'period']).agg({'monitoring_class':'count'})

# Primeiro, calculamos a contagem
count_df = gdf.groupby(['dataset_part', 'period']).agg(count=('monitoring_class', 'count'))

# Em seguida, calculamos a porcentagem
total_count = count_df['count'].sum()
count_df['percentage'] = (count_df['count'] / total_count) * 100

In [51]:
count_df

count  percentage
dataset_part period                      
test         2017/2018     32    0.083787
             2018/2019     51    0.133536
             2019/2020     90    0.235651
             2020/2021   2696    7.059070
             2021/2022   2868    7.509426
train        2017/2018    144    0.377042
             2018/2019    238    0.623167
             2019/2020    416    1.089233
             2020/2021  12481   32.679619
             2021/2022  13447   35.208944
val          2017/2018     30    0.078550
             2018/2019     52    0.136154
             2019/2020     89    0.233033
             2020/2021   2657    6.956954
             2021/2022   2901    7.595832